<link rel='stylesheet' href='../assets/css/main.css'/>

[<< back to main index](../README.md)

# Multiple Linear Regression Lab 5 : House Prices

### Overview
Estimate house prices using MLR

### Builds on
None

### Run time
approx. 20 minutes

### Notes



In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler



## Step 1 : House data

In [ ]:
housePrices = spark.read.csv("/data/house-prices/house-sales-full.csv", header=True, inferSchema=True)
housePrices.show()

Wow! That's a lot of columns.  Maybe we should foucs on just a few of them.


In [ ]:

housePrices_small = housePrices.select("SalePrice", "Bedrooms", "Bathrooms", "SqFtTotLiving", "SqFtLot")
housePrices_small.show()

## Step 2: Create Features and Label Column in Dataframe

We will create our "features" column, which is of type vector, and our label column, for which we can just reuse the SquareFootage column.

In [ ]:
assembler = VectorAssembler(inputCols=["Bedrooms", "Bathrooms", "SqFtTotLiving", "SqFtLot"], outputCol="features")
featureVector = assembler.transform(housePrices_small)

featureVector = featureVector.withColumnRenamed("SalePrice", "label")
featureVector.show()

In [ ]:
## Step 3: Run the Linear Regression Model
lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
lrModel = lr.fit(featureVector)

print("Coefficents:" + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))



In [ ]:
print("numIterations: %d" % lrModel.summary.totalIterations)
print("objectiveHistory: %s" % str(lrModel.summary.objectiveHistory))
lrModel.summary.residuals.show()
print("RMSE: %f" % lrModel.summary.rootMeanSquaredError)
print("r2: %f" % lrModel.summary.r2)


**Observe the summary output**

## Step 3:  Predict some home prices
Create a new data frame with the following data

In [ ]:

# Make a Pandas dataframe
new_data = pd.DataFrame({'Bedrooms' : [5,3,2],
                         'Bathrooms' : [3,2,1.5],
                         'SqFtTotLiving' : [4400, 1800, 1550],
                         'SqFtLot' : [10000, 5000, 4000]
             })

new_data_spark = spark.createDataFrame(new_data)
new_featureVector = assembler.transform(new_data_spark)  #run VectorAssembler again to create features.
new_featureVector.show()




## Step 4: Run on some prediction data

Lets see the predicted output.

In [ ]:
predicted_prices = lrModel.transform(new_featureVector)
predicted_prices.show()